In [1]:
import pickle 
import numpy as np
import pandas as pd
import statistics
from PIL import Image
import os 
import random
import time
import matplotlib.pyplot as plt
import netaddr

In [2]:
import dgl
import torch
import torch.nn as nn
import dgl.function as fn
import dgl.nn as dglnn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [3]:
#pip install netaddr

In [4]:
#directory should change based on the device, for laptop it should be users//jalal for office desktop it is users//jghadermazi

In [5]:
wed_data = pd.read_pickle(r'C:\\Users\\Jalal\\Box\\GAN Project\\Sample data\\Wednesday_Packet_Level_Data.pkl')

In [6]:
df1 = wed_data.copy()

In [7]:
del wed_data

In [8]:
df1[['Payload']] = df1[['Payload']].replace('', '00')

In [9]:
df1

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,1499258820.014743000,23.60.139.27,192.168.10.17,80,47757,12035,1,0023ae9b956700c1b114eb310800,45000034a7b6400037062efd173c8b1bc0a80a11,0050ba8da529957b5d3f78e3801003ab15920000010108...,00,fwd,1.0,1.0
1,1499258820.039091000,23.60.139.27,192.168.10.17,80,47757,12035,1,0023ae9b956700c1b114eb310800,45000722a7b740003706280e173c8b1bc0a80a11,0050ba8da529957b5d3f78e3801803ab74250000010108...,485454502f312e3120323030204f4b0d0a536572766572...,bwd,1.0,1.0
2,1499258820.039116000,192.168.10.17,23.60.139.27,47757,80,12035,435,00c1b114eb310023ae9b95670800,45000034fcd740004006d0dbc0a80a11173c8b1b,ba8d00505d3f78e3a5299c698010010011280000010108...,00,fwd,1.0,1.0
3,1499258820.082489000,192.168.10.12,208.185.50.90,51774,443,11808,4580,00c1b114eb31b8ac6f3604e30800,4500002804414000400668c7c0a80a0cd0b9325a,ca3e01bb4c69a4a67cf3967050109eb472ea0000,000000000000,fwd,1.0,1.0
4,1499258820.084153000,192.168.10.17,23.208.97.23,37115,443,12034,349,00c1b114eb310023ae9b95670800,4500017a021640004006f3c7c0a80a1117d06117,90fb01bb96268b0ed08217e7801801356d9e0000010108...,1703030141000000000000000174b7a44755da49b50c35...,fwd,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11334902,1499276130.701758000,172.16.0.1,192.168.10.51,443,53199,243533,0,b8ac6f360ba800c1b114eb310800,4500003cc45240003f06007dac100001c0a80a33,01bbcfcf167451881e1ff3d6a012fe884e0a0000020405...,00,bwd,0.0,0.0
11334903,1499276130.702180000,172.16.0.1,192.168.10.51,443,53199,243533,1,b8ac6f360ba800c1b114eb310800,45000034034d40003f06c18aac100001c0a80a33,01bbcfcf167451891e1ff5db8010000879580000010108...,00,bwd,0.0,0.0
11334904,1499276130.702964000,172.16.0.1,192.168.10.51,443,53199,243533,1,b8ac6f360ba800c1b114eb310800,450000bd034e40003f06c100ac100001c0a80a33,01bbcfcf167451891e1ff5db801800084ed60000010108...,16030300510200004d0303587c9e338fe3870b8fd3c17c...,bwd,0.0,0.0
11334905,1499276130.710822000,172.16.0.1,192.168.10.51,443,53199,243533,138,b8ac6f360ba800c1b114eb310800,45000034034f40003f06c188ac100001c0a80a33,01bbcfcf167452121e1ffa1f80100008748a0000010108...,00,bwd,0.0,0.0


In [10]:
#benign    0
#dos slowloris    1
#dos slowhttptest   2
#dos hulk     3
#dos goldeneye    4
#heartbleed    5
#ftp patator    6
#ssh patator    7
#webattack bruteforce    8
#webattack xss     9
#webattack sql injection 10
#infiltration     11
#botnet     12
#portscan   13
#ddos     14

In [11]:
df1['Class Label'].value_counts()

0.0    8845368
3.0    2246818
4.0     106177
5.0      49296
1.0      47586
2.0      39662
Name: Class Label, dtype: int64

In [12]:
df = df1[df1['Class Label']== 4]

In [13]:
df = df.reset_index(inplace = False)

In [14]:
df

,index,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,2334066,1499263038.586094000,192.168.10.50,172.16.0.1,80,50432,164463,8689,00c1b114eb310019b90a69f10800,45000b8f41ad4000400676d0c0a80a32ac100001,0050c500afc3f0ef0b0535cc801900eb826d0000010108...,2020202020202020202020202020202020202020202020...,fwd,1.0,4.0
1,2334067,1499263038.586613000,172.16.0.1,192.168.10.50,50404,80,164451,365,0019b90a69f100c1b114eb310800,45000028089740003e06bd4dac100001c0a80a32,c4e40050c579628f00000000500400004bb70000,000000000000,bwd,1.0,4.0
2,2334068,1499263038.586629000,192.168.10.50,172.16.0.1,80,50510,164544,1,00c1b114eb310019b90a69f10800,4500003485a8400040063e30c0a80a32ac100001,0050c54eba4b0acd481abb2e801000ebe3c80000010108...,00,fwd,1.0,4.0
3,2334069,1499263038.586631000,172.16.0.1,192.168.10.50,50532,80,164574,0,0019b90a69f100c1b114eb310800,4500003c524240003e06738eac100001c0a80a32,c56400505622a95300000000a00272109ec30000020405...,00,fwd,1.0,4.0
4,2334070,1499263038.586631000,172.16.0.1,192.168.10.50,50412,80,164453,325,0019b90a69f100c1b114eb310800,45000034383140003e068da7ac100001c0a80a32,c4ec0050816ac46d3c8d55678014019ad4100000010108...,00,fwd,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106172,2440238,1499263072.330652000,172.16.0.1,192.168.10.50,34026,80,170656,0,0019b90a69f100c1b114eb310800,4500003c469140003e067f3fac100001c0a80a32,84ea005003695a9400000000a00272105fc20000020405...,00,bwd,1.0,4.0
106173,2440239,1499263072.330656000,172.16.0.1,192.168.10.50,59080,80,170485,362,0019b90a69f100c1b114eb310800,450000344a1040003e067bc8ac100001c0a80a32,e6c80050965048b6e222237b8010013f63d20000010108...,00,fwd,1.0,4.0
106174,2440240,1499263072.330656000,172.16.0.1,192.168.10.50,33868,80,170505,396,0019b90a69f100c1b114eb310800,45000034df7540003e06e662ac100001c0a80a32,844c0050e6adc60e4e5c7ab08010011235560000010108...,00,bwd,1.0,4.0
106175,2440241,1499263072.330657000,172.16.0.1,192.168.10.50,59080,80,170485,362,0019b90a69f100c1b114eb310800,450000344a1140003e067bc7ac100001c0a80a32,e6c80050965048b6e222237b8014013f63cd0000010108...,00,fwd,1.0,4.0


In [15]:
ip_header_lst = []

for i in range(len(df)):
    x = df['IP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 9, 12, 13, 14, 15, 16, 17, 18, 19])
    
    output = np.abs(output.astype(float))/255
    ip_header_lst.append(output)
    
df['IP Header Array'] = ip_header_lst

In [16]:
tcp_header_lst = []

for i in range(len(df)):
    x = df['TCP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 2, 3])
    output = np.abs(output.astype(float))/255
    
    tcp_header_lst.append(output)
    
df['TCP Header Array'] = tcp_header_lst

In [17]:
df['pkt_header_values'] = df.apply(lambda x : (np.append(x['IP Header Array'], x['TCP Header Array'])), axis=1)

In [18]:
new_cols = ['Source IP', 'Dst IP', 'Source Port', 'Dst Port','Epoch Time','Stream Index', 'Sequence Number', 'Direction', 'pkt_header_values']
df = df[new_cols]

In [19]:
# 70% of benign data for train, 30% for test benign

In [20]:
#random.seed(0)
#mlist = random.sample(streams_lst, 61492)

In [21]:
#len(imlist)

In [22]:
#df_train = df.loc[df['Stream Index'].isin(imlist)]
#df_train = df_train.reset_index(inplace = False)

In [23]:
#df_test = df.loc[~df['Stream Index'].isin(imlist)]
#df_test = df_test.reset_index(inplace = False)

In [24]:
#streams_lst1 = list(df_test['Stream Index'].unique())

In [25]:
#len(streams_lst1)

In [26]:
df

,Source IP,Dst IP,Source Port,Dst Port,Epoch Time,Stream Index,Sequence Number,Direction,pkt_header_values
0,192.168.10.50,172.16.0.1,80,50432,1499263038.586094000,164463,8689,fwd,"[0.043137254901960784, 0.5607843137254902, 0.2..."
1,172.16.0.1,192.168.10.50,50404,80,1499263038.586613000,164451,365,bwd,"[0.0, 0.1568627450980392, 0.03137254901960784,..."
2,192.168.10.50,172.16.0.1,80,50510,1499263038.586629000,164544,1,fwd,"[0.0, 0.20392156862745098, 0.5215686274509804,..."
3,172.16.0.1,192.168.10.50,50532,80,1499263038.586631000,164574,0,fwd,"[0.0, 0.23529411764705882, 0.3215686274509804,..."
4,172.16.0.1,192.168.10.50,50412,80,1499263038.586631000,164453,325,fwd,"[0.0, 0.20392156862745098, 0.2196078431372549,..."
...,...,...,...,...,...,...,...,...,...
106172,172.16.0.1,192.168.10.50,34026,80,1499263072.330652000,170656,0,bwd,"[0.0, 0.23529411764705882, 0.27450980392156865..."
106173,172.16.0.1,192.168.10.50,59080,80,1499263072.330656000,170485,362,fwd,"[0.0, 0.20392156862745098, 0.2901960784313726,..."
106174,172.16.0.1,192.168.10.50,33868,80,1499263072.330656000,170505,396,bwd,"[0.0, 0.20392156862745098, 0.8745098039215686,..."
106175,172.16.0.1,192.168.10.50,59080,80,1499263072.330657000,170485,362,fwd,"[0.0, 0.20392156862745098, 0.2901960784313726,..."


In [27]:
streams_lst = list(df['Stream Index'].unique())
print('Total number of flows in class:', len(streams_lst))    
random.shuffle(streams_lst)

# Directory path where to save the CSV files
directory_path = 'Wednesday_goldeneye_all/'

for PKT in range(2, 16):
    
    file_name = f'wednesday_goldeneye_{PKT}pkt.csv'
    file_path = directory_path + file_name
    

    data = []
    num_pkt = PKT
    pkt_val_pad = [0] * 25


    for j in range(len(streams_lst)):

        df_stream = df[df['Stream Index'] == streams_lst[j]]


        df_stream = df_stream.sort_values(by = 'Epoch Time')
        df_stream = df_stream.reset_index(drop=True)


        if len(df_stream) >= num_pkt:

            src_node = (int(netaddr.IPAddress(df_stream['Source IP'][0])), int(df_stream['Source Port'][0]))
            dst_node = (int(netaddr.IPAddress(df_stream['Dst IP'][0])), int(df_stream['Dst Port'][0]))
            stream_no = df_stream['Stream Index'][0]

            pkt_val_seq = []

            for k in range(0, num_pkt):

                pkt_val = []
                if df_stream['Direction'][k] == 'fwd':
                    pkt_val = [*df_stream['pkt_header_values'][k], *pkt_val_pad]
                else:
                    pkt_val = [*pkt_val_pad, *df_stream['pkt_header_values'][k]]

                pkt_val_seq.extend(pkt_val)

            data.append([src_node, dst_node, stream_no, pkt_val_seq])

    df2 = pd.DataFrame(data, columns= ['src_node', 'dst_node', 'stream_number', 'pkt_vals'])
    
    df2.to_csv(file_path, index=False)

Total number of flows in class: 6704
